In [1]:
import pandas as pd
import numpy as np
import hashlib

In [2]:
# Read file
df = pd.read_csv("datasets/raw/insurance_undertakings-raw.csv", sep=";", dtype=object)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30861 entries, 0 to 30860
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   International Name                    30861 non-null  object
 1   Name of NCA                           30861 non-null  object
 2   Home Country                          30861 non-null  object
 3   Identification code                   30861 non-null  object
 4   LEI                                   24180 non-null  object
 5   Official name of the entity           30861 non-null  object
 6   Cross border status                   30861 non-null  object
 7   EU Country where the entity operates  30861 non-null  object
 8   Local/Commercial name                 16594 non-null  object
 9   Registration start date               30861 non-null  object
 10  Registration end date                 128 non-null    object
 11  Street Name                 

In [3]:
# Drop empty columns
df = df.drop(["Unnamed: 19", "Operation End Date"], axis=1)

# Set LEI to NaN if filled with zeros
df.loc[df["LEI"] == "00000000000000000000", "LEI"] = None

# Convert colums to date time
df["Registration start date"] = pd.to_datetime(df["Registration start date"])
df["Registration end date"] = pd.to_datetime(df["Registration end date"])
df["Operation Start Date"] = pd.to_datetime(df["Operation Start Date"])

In [4]:
df.head()

,International Name,Name of NCA,Home Country,Identification code,LEI,Official name of the entity,Cross border status,EU Country where the entity operates,Local/Commercial name,Registration start date,Registration end date,Street Name,Street Number,Post Code,City,County,Website address,Operation Start Date
0,EUROPEAN INSURANCE SOLUTION PCC LIMITED,Malta Financial Services Authority (MFSA),MT,C 62414,213800LD98RC6499PB21,EUROPEAN INSURANCE SOLUTION PCC LIMITED,Domestic undertaking,MT,NaN,2013-11-14 01:00:00,NaT,"St. Anne Street,","4th Floor, Development House,",FRN 9010,Floriana,NaN,NaN,2013-11-14 01:00:00
1,ELMO INSURANCE LIMITED,Malta Financial Services Authority (MFSA),MT,C 3500,213800PTT2RWKYU5LV82,ELMO INSURANCE LIMITED,Domestic undertaking,MT,NaN,2004-04-29 02:00:00,NaT,Abate Rigord Street,NaN,XBX 1111,Ta Xbiex,NaN,NaN,2004-04-29 02:00:00
2,CITADEL INSURANCE P.L.C.,Malta Financial Services Authority (MFSA),MT,C 21550,5299005JR2DPHSDP1371,CITADEL INSURANCE P.L.C.,Domestic undertaking,MT,NaN,1997-08-22 02:00:00,NaT,"26, Market Street,","Casa Borgo,",FRN 1082,Floriana,NaN,NaN,1997-08-22 02:00:00
3,BAVARIA REINSURANCE MALTA LIMITED,Malta Financial Services Authority (MFSA),MT,C 37534,213800VP6B1WEHBDQP63,BAVARIA REINSURANCE MALTA LIMITED,Domestic undertaking,MT,NaN,2005-12-14 01:00:00,NaT,"St. Anne Street,","5th Floor, Development House,",FRN 9010,Floriana,NaN,NaN,2005-12-14 01:00:00
4,CAFINA ASSURANCES LIMITED,Malta Financial Services Authority (MFSA),MT,C 50718,213800NR2H6HYASL4F61,CAFINA ASSURANCES LIMITED,Domestic undertaking,MT,NaN,2010-10-15 02:00:00,NaT,"Alfred Craig Street,",NaN,XBX 1111,Ta Xbiex,NaN,NaN,2010-10-15 02:00:00


In [5]:
# Check columns
df.columns

Index(['International Name', 'Name of NCA', 'Home Country',
       'Identification code', 'LEI', 'Official name of the entity',
       'Cross border status', 'EU Country where the entity operates',
       'Local/Commercial name', 'Registration start date',
       'Registration end date', 'Street Name', 'Street Number', 'Post Code',
       'City', 'County', 'Website address', 'Operation Start Date'],
      dtype='object')

In [6]:
# Note that not all entities in the list have a Legal Entity Identifier - for those that don't use the international name as proxy key
# In either case hash the LEI and Name to get a consistent key
df["Key"] = np.where(df["LEI"].isnull(), df["International Name"].apply(lambda x: hashlib.md5(str(x).encode('UTF-8')).hexdigest()).str[:12], 
                                         df["LEI"].apply(lambda x: hashlib.md5(str(x).encode('UTF-8')).hexdigest()).str[:12])

# If duplicate keys are present prefer keeping those entries with a website
df_insureres = df.sort_values(["Key", "Website address"]).reset_index(drop=True).drop_duplicates(subset=["Key"], keep="first")

In [7]:
# Keep selected columns
df_insureres = df_insureres[["Key", "LEI", "International Name", "Official name of the entity", "Name of NCA", "Registration start date", "Registration end date", "Street Name", "Street Number", "Post Code", "City", "County", "Home Country", "Website address", "Operation Start Date"]]

# Get list of countries in which insurer is operating and turn it into a nested array
df_countries = df[["Key", "EU Country where the entity operates"]].drop_duplicates()
df_countries = df_countries.groupby(["Key"])["EU Country where the entity operates"].apply(list).reset_index()

# Combine insurer list with country of operation
df_insureres = df_insureres.merge(df_countries, on=["Key"])

df_insureres.head()


,Key,LEI,International Name,Official name of the entity,Name of NCA,Registration start date,Registration end date,Street Name,Street Number,Post Code,City,County,Home Country,Website address,Operation Start Date,EU Country where the entity operates
0,000a66771871,635400AVUOEWJYZGQQ40,Lion II Re Designated Activity Company,Lion II Re Designated Activity Company,Central Bank of Ireland,2017-06-14 02:00:00,NaT,"Floor 4, 25-28 Adelaide Road",NaN,D2,Dublin,Dublin 2,IE,NaN,2017-06-14 02:00:00,[IE]
1,001730d96ec4,9598001DQ7U3BQJEEZ51,"AGROPELAYO SOCIEDAD DE SEGUROS, SOCIEDAD ANÓNIMA","AGROPELAYO SOCIEDAD DE SEGUROS, SOCIEDAD ANÓNIMA",DGSFP,2015-12-18 01:00:00,NaT,Cl Micer Mascó,42,46010,Valencia,Valencia,ES,NaN,2015-12-18 01:00:00,[ES]
2,0030ac7998cd,391200DTAYLSAHINXK49,Tokio Marine Kiln Insurance Limited,Tokio Marine Kiln Insurance Limited,Prudential Regulation Authority,1970-09-15 02:00:00,NaT,Fenchurch Street,20,EC3M 3BY,London,NaN,UK,www.tokiomarine.co.uk,2001-01-12 01:00:00,[UK]
3,00371fcc8790,743700IXQUK2UEMLWB77,Försäkringsaktiebolaget Liv-Alandia,Försäkringsaktiebolaget Liv-Alandia,FIN-FSA,1994-05-10 02:00:00,NaT,Ålandsvägen 31,NaN,22100,MARIEHAMN,NaN,FI,http://www.alandia.com,1995-05-30 02:00:00,[FI]
4,003f3f91d298,959800ZWWVLBYKM42V32,"MERIDIANO, S.A., COMPAÑIA ESPAÑOLA DE SEGUROS","MERIDIANO, S.A., COMPAÑIA ESPAÑOLA DE SEGUROS",DGSFP,1967-07-24 02:00:00,NaT,Av JEAN LAUDE COMBALDIEU,5,03008,Alicante/Alacant,Alicante,ES,NaN,1967-07-24 02:00:00,[ES]


In [8]:
# Save to .csv
df_insureres.to_csv("datasets/clean/european-insurers.csv", index=False)